In [76]:
import openmc
import openmc.data as data
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp

# Helper Functions

In [100]:
def at2wt(*args):
    wt_dict = {}
    for iso, at in args:
        mass = data.atomic_mass(iso)
        wt_dict[iso] = at * mass
        
    denom = sum(wt_dict.values())
    
    for iso in wt_dict.keys():
        wt_dict[iso] /= denom
    return wt_dict 

def alpha(T):
    return -0.4247 + 1.282e-3 * T + 7.362e-7 * T**2 - 2.069e-10 * T**3

def density(T, rho0, t0 = 25):
    differential = T - t0
    return rho0 / (1 + 3 * alpha(T) * differential)

def fuel_density(T):
    U_rho = 19360 - 1.03347 * T
    Zr_rho = 6550 - 0.1685 * T
    atZr = .2245
    return atZr * Zr_rho + (1-atZr) * U_rho

def sodium_density(T):
    temps = np.array([126.85, 226.85, 326.85, 426.85, 526.85, 626.85, 726.85])
    rhos = np.array([919, 897, 874, 852, 828, 805, 781])
    rho_func = scp.interpolate.interp1d(temps,rhos)
    return np.float64(rho_func(T))

# Materials

In [105]:
def fuel(at, suffix='',temp = 507.5):
    
    u235 = ('U235', at)
    u238 = ('U238', 1-at)
    wt = at2wt(u235, u238)['U235']
    
    mat = openmc.Material(name=f'fuel_{suffix}', temperature=temp)
    
    mat.add_element('Zr', .1, 'wo')
    mat.add_nuclide('U235', 0.9*wt, 'wo')
    mat.add_nuclide('U238', 0.9*(1-wt), 'wo')
    mat.set_density('kg/m3', fuel_density(temp))
    return mat

def sodium(temp, name):
    mat = openmc.Material(name=name, temperature=temp)
    
    mat.add_element('Na', 1.0)
    mat.set_density('kg/m3', sodium_density(temp))
    return mat

def cladding(temp):
    mat = openmc.Material(name='cladding', temperature=temp)
    
    mat.add_element('Fe', 85.007, 'wo')
    mat.add_element('Cr', 11.79 , 'wo')
    mat.add_element('Mo', 0.99  , 'wo')
    mat.add_element('Mn', 0.59  , 'wo')
    mat.add_element('Ni', 0.53  , 'wo')
    mat.add_element('W' , 0.45  , 'wo')
    mat.add_element('V' , 0.31  , 'wo')
    mat.add_element('C' , 0.19  , 'wo')
    mat.add_element('Si', 0.10  , 'wo')
    mat.add_element('Nb', 0.02  , 'wo')
    mat.add_element('P' , 0.019 , 'wo')
    mat.add_element('N' , 0.01  , 'wo')
    mat.add_element('S' , 0.004 , 'wo')

    rho0 = 7700.0
    mat.set_density('kg/m3', density(temp, rho0))
    return mat

def control_rod(temp):
    mat = openmc.Material(name='control_rod', temperature=temp)

    mat.add_elements_from_formula('C4B')

    rho0 = 2520
    mat.set_density('kg/m3', density(temp, rho0))
    return mat

In [107]:
inner_fuel = fuel(.109, 'inner')
middle_fuel = fuel(.124, 'middle')
outer_fuel = fuel(.155, 'outer')

bond = sodium(497.5, 'bond')
coolant = sodium(432.5, 'coolant')

clad = cladding(482.5)

cntrl_rod = control_rod(450.0) # dont know cr temp

MATERIALS = openmc.Materials([inner_fuel, middle_fuel, outer_fuel, bond, coolant, clad, cntrl_rod])